<a href="https://colab.research.google.com/github/Khozinkosi/Comp702/blob/main/Copy_of_Assignment3Comp703.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   Wandile Ngobese (222056013)
*   Linda Sekhoasha (222004139)
*   Khonzinkosi Mkhize (219005273)
*   Samukelo Mkhize (220009930)


In [ ]:
!pip install nltk

In [ ]:
!pip install tqdm

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Bidirectional
import matplotlib.pyplot as plt
import os
import zipfile
import urllib.request
import re
import nltk
from nltk.translate.bleu_score import sentence_bleu
nltk.download('punkt')  # Required for tokenization
from tqdm import tqdm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Load and Process dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = '/content/drive/My Drive/fra-eng/fra.txt'

In [ ]:
with open(dataset_path, 'r', encoding='utf-8') as file:
    lines = file.read().strip().split('\n')

In [ ]:
input_texts = []
target_texts = []

for line in lines:
    parts = line.strip().split('\t')
    eng, fra = parts[0], parts[1]
    input_texts.append(eng)
    target_texts.append('\t' + fra + '\n')  # Add start '\t' and end '\n' tokens

In [ ]:
for i in range(-5, 0):
    print(f"{i+len(input_texts)}: {input_texts[i]}")


237833: Death is something that we're often discouraged to talk about or even think about, but I've realized that preparing for death is one of the most empowering things you can do. Thinking about death clarifies your life.
237834: Since there are usually multiple websites on any given topic, I usually just click the back button when I arrive on any webpage that has pop-up advertising. I just go to the next page found by Google and hope for something less irritating.
237835: If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker.
237836: It may be impossible to get a completely error-free corpus due to the nature of this kind of collaborative effort. However, if we encourage members to contribute sentences in their own languages rather than experiment in languages they are learning

In [ ]:
for i in range(5):
    print(f"English: {input_texts[i]}")
    print(f"French : {target_texts[i]}")
    print('---')

English: Go.
French : 	Va !

---
English: Go.
French : 	Marche.

---
English: Go.
French : 	En route !

---
English: Go.
French : 	Bouge !

---
English: Hi.
French : 	Salut !

---


In [ ]:
input_characters = sorted(set(char for text in input_texts for char in text))
target_characters = sorted(set(char for text in target_texts for char in text))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max(len(text) for text in input_texts)
max_decoder_seq_length = max(len(text) for text in target_texts)


**Then create mappings:**

In [ ]:
input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}


** Create integer-encoded sequences**

In [ ]:
input_token_index = {char: i + 1 for i, char in enumerate(input_characters)}  # reserve 0 for padding
target_token_index = {char: i + 1 for i, char in enumerate(target_characters)}

encoder_input_seq = []
decoder_input_seq = []
decoder_target_seq = []

for i in range(len(input_texts)):
    input_seq = [input_token_index[char] for char in input_texts[i]]
    target_seq = [target_token_index[char] for char in target_texts[i]]

    decoder_input = target_seq[:-1]
    decoder_target = target_seq[1:]

    encoder_input_seq.append(input_seq)
    decoder_input_seq.append(decoder_input)
    decoder_target_seq.append(decoder_target)


**Create one-hot encoded 3D arrays for encoder input, decoder input, and decoder target:**

In [ ]:
'''import numpy as np

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        #encoder_input_data[i, t, input_token_index[char]] = 1.0
        num_encoder_tokens = len(input_token_index)  # Should be correct

    for t, char in enumerate(target_text):
        target_token_index = {char: i + 1 for i, char in enumerate(target_characters)}
        num_decoder_tokens = len(target_token_index) + 1  # +1 is CRUCIAL!



        if t > 0:
            # decoder target data is ahead by one timestep
            target_token_index = {char: i + 1 for i, char in enumerate(target_characters)}  # Start from 1'''


import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# -----------------------------
# Step 1: Build token dictionaries
# -----------------------------
input_token_index = {char: i + 1 for i, char in enumerate(input_characters)}  # start from 1
target_token_index = {char: i + 1 for i, char in enumerate(target_characters)}  # start from 1

num_encoder_tokens = len(input_token_index) + 1  # +1 for padding (index 0)
num_decoder_tokens = len(target_token_index) + 1

# -----------------------------
# Step 2: Convert characters to integer sequences
# -----------------------------
encoder_input_sequences = []
decoder_input_sequences = []
decoder_target_sequences = []

for i in range(len(input_texts)):
    input_seq = [input_token_index[char] for char in input_texts[i]]
    target_seq = [target_token_index[char] for char in target_texts[i]]

    encoder_input_sequences.append(input_seq)
    decoder_input_sequences.append(target_seq[:-1])  # decoder input excludes last char
    decoder_target_sequences.append(target_seq[1:])  # decoder target excludes first char

# -----------------------------
# Step 3: Pad sequences
# -----------------------------
encoder_input_data = pad_sequences(encoder_input_sequences, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences(decoder_input_sequences, maxlen=max_decoder_seq_length, padding='post')
decoder_target_data = pad_sequences(decoder_target_sequences, maxlen=max_decoder_seq_length, padding='post')

# -----------------------------
# Step 4: Reshape decoder target for training
# -----------------------------
decoder_target_data = np.expand_dims(decoder_target_data, -1)

# -----------------------------
# Step 5: Define the model with Embeddings
# -----------------------------
latent_dim = 128  # Feel free to adjust based on your system

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=num_encoder_tokens, output_dim=64)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(latent_dim, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=num_decoder_tokens, output_dim=64)(decoder_inputs)
decoder_lstm, _, _ = LSTM(latent_dim, return_sequences=True, return_state=True)(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm)

# Combined Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 64)  │      5,824 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 64)  │      7,424 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 128),     │     98,816 │ embedding[0][0]   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │     98,816 │ embedding_1[0][0… │
│                     │ 128), (None,      │            │ lstm[0][1],       │
│                     │ 128), (None,      │            │ lstm[0][2]        │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 116) │     14,964 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 225,844 (882.20 KB)

 Trainable params: 225,844 (882.20 KB)

 Non-trainable params: 0 (0.00 B)

Bidirectional LSTM Encoder + LSTM Decoder

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional

# Set latent dimensionality
latent_dim = 128  # You can adjust this based on memory

# Define encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = Bidirectional(LSTM(latent_dim, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_inputs)
state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Full model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None, 91)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, 256),     │    225,280 │ input_layer_2[0]… │
│ (Bidirectional)     │ (None, 128),      │            │                   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, None, 116) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, None,     │    381,952 │ input_layer_3[0]… │
│                     │ 256), (None,      │            │ concatenate[0][0… │
│                     │ 256), (None,      │            │ concatenate_1[0]… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None, 116) │     29,812 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 637,044 (2.43 MB)

 Trainable params: 637,044 (2.43 MB)

 Non-trainable params: 0 (0.00 B)

**imit Number of Samples **

In [ ]:
MAX_SAMPLES = 5000  # or start with 20000
filtered_input = []
filtered_target = []

for line in lines:
    parts = line.strip().split('\t')
    if len(parts) == 2:
        eng, fra = parts
        filtered_input.append(eng)
        filtered_target.append('\t' + fra + '\n')
    if len(filtered_input) >= MAX_SAMPLES:
        break

input_texts = filtered_input
target_texts = filtered_target


In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=num_encoder_tokens, output_dim=64)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(128, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=num_decoder_tokens, output_dim=64)(decoder_inputs)
decoder_lstm, _, _ = LSTM(128, return_sequences=True, return_state=True)(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm)

# Model
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Train the Model

In [ ]:
'''model.fit([encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=64,
          epochs=10,
          validation_split=0.2)
#model.save("seq2seq_lstm.h5")'''


import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
from sklearn.model_selection import train_test_split

# ========== CONFIGURATION ==========
latent_dim = 128
num_samples = 1000  # Use a subset for testing
epochs = 5
batch_size = 32

# ========== SAMPLE DATA ==========
input_texts = ["hello"] * num_samples
target_texts = ["\tbonjour\n"] * num_samples  # \t = start, \n = end

input_characters = sorted(list(set("".join(input_texts))))
target_characters = sorted(list(set("".join(target_texts))))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

# ========== VECTORIZE ==========
encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

# ========== TRAIN-TEST SPLIT ==========
X_train_enc, X_test_enc, X_train_dec, X_test_dec, y_train, y_test = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data, test_size=0.2, random_state=42
)

# ========== ENCODER ==========
encoder_inputs = Input(shape=(None, num_encoder_tokens), name="encoder_inputs")
encoder_bi_lstm = Bidirectional(LSTM(latent_dim, return_state=True, return_sequences=False), name="bidirectional_lstm")
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bi_lstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

# ========== DECODER ==========
decoder_inputs = Input(shape=(None, num_decoder_tokens), name="decoder_inputs")
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax", name="decoder_dense")
decoder_outputs = decoder_dense(decoder_outputs)

# ========== MODEL ==========
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# ========== TRAIN ==========
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(
    [X_train_enc, X_train_dec],
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1
)

# ========== EVALUATE ==========
loss, accuracy = model.evaluate([X_test_enc, X_test_dec], y_test)
print(f"\n✅ Test Loss: {loss:.4f} — Test Accuracy: {accuracy:.4f}")


Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.3259 - loss: 1.6475 - val_accuracy: 0.7778 - val_loss: 0.8167
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8254 - loss: 0.5725 - val_accuracy: 0.8889 - val_loss: 0.1493
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8889 - loss: 0.1119 - val_accuracy: 0.8889 - val_loss: 0.0481
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8889 - loss: 0.0414 - val_accuracy: 0.8889 - val_loss: 0.0286
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8889 - loss: 0.0264 - val_accuracy: 0.8889 - val_loss: 0.0217
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8889 - loss: 0.0217 

✅ Test Loss: 0.0217 — Test Accuracy: 0.8889


**Bidirectional LSTM Encoder + LSTM Decoder**

**Create a function that evaluates your model's translations using the BLEU score.**

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def decode_sequence(input_seq, encoder_model, decoder_model, max_decoder_seq_length, target_token_index, reverse_target_char_index):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence with just the start character.
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']

    decoded_sentence = ''
    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')

        if sampled_char == '\n':
            break
        decoded_sentence += sampled_char

        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence

# Placeholder: define encoder_model and decoder_model later (after training)


**Set up inference models (separate encoder and decoder) for testing.**

**This function uses the inference models to translate an English sentence (as a character sequence) into French.**

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
from keras.optimizers import Adam

# Dummy data for example - replace with your own
num_samples = 1000
latent_dim = 128
input_texts = ["hello"] * num_samples
target_texts = ["\tbonjour\n"] * num_samples

input_characters = sorted(list(set("".join(input_texts))))
target_characters = sorted(list(set("".join(target_texts))))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

# --- Encoder ---
encoder_inputs = Input(shape=(None, num_encoder_tokens), name="encoder_inputs")
encoder_bi_lstm = Bidirectional(LSTM(latent_dim, return_state=True), name="bidirectional_lstm")
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bi_lstm(encoder_inputs)

# Merge forward and backward states
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# Reduce back to latent_dim using Dense layers
reduced_state_h = Dense(latent_dim, activation="tanh")(state_h)
reduced_state_c = Dense(latent_dim, activation="tanh")(state_c)
encoder_states = [reduced_state_h, reduced_state_c]

# --- Decoder ---
decoder_inputs = Input(shape=(None, num_decoder_tokens), name="decoder_inputs")
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax", name="decoder_dense")
decoder_outputs = decoder_dense(decoder_outputs)

# --- Model ---
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=10,
    validation_split=0.2
)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - accuracy: 0.2496 - loss: 1.7799 - val_accuracy: 0.3333 - val_loss: 1.4084
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.4203 - loss: 1.2206 - val_accuracy: 0.6667 - val_loss: 0.7994
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8051 - loss: 0.7102 - val_accuracy: 0.8889 - val_loss: 0.4713
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8889 - loss: 0.4067 - val_accuracy: 0.8889 - val_loss: 0.2424
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8889 - loss: 0.2109 - val_accuracy: 0.8889 - val_loss: 0.1351
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8889 - loss: 0.1208 - val_accuracy: 0.8889 - val_loss: 0.0866
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8889 - loss: 0.0806 - val_accuracy: 0.8889 - val_loss: 0.0664
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8889 - loss: 0.0631 - val_accuracy: 0.8889 - v

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence with just the start character '\t'
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']

    # Initialize decoded sentence
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')

        decoded_sentence += sampled_char

        # Exit if end token or max length reached
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


**Create an evaluation wrapper with BLEU score**

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
smoothie = SmoothingFunction().method4

def evaluate_bleu(n=100):
    total_bleu = 0.0
    for i in range(n):
        input_seq = encoder_input_data[i:i+1]  # One sample
        decoded_sentence = decode_sequence(input_seq)

        # Remove start/end tokens
        reference = [list(target_texts[i][1:-1])]
        candidate = list(decoded_sentence.strip())

        bleu = sentence_bleu(reference, candidate, smoothing_function=smoothie)
        total_bleu += bleu

        print(f'Input: {input_texts[i]}')
        print(f'Target: {target_texts[i]}')
        print(f'Predicted: {decoded_sentence.strip()}')
        print(f'BLEU: {bleu:.4f}')
        print('---')

    avg_bleu = total_bleu / n
    print(f'\nAverage BLEU score over {n} samples: {avg_bleu:.4f}')


**Run the evaluation**

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input

# Create inference encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder inference setup
decoder_state_input_h = Input(shape=(latent_dim,), name="decoder_state_input_h")
decoder_state_input_c = Input(shape=(latent_dim,), name="decoder_state_input_c")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to characters
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

# Decoding function
def decode_sequence(input_seq):
    # Encode the input
    states_value = encoder_model.predict(input_seq)

    # Start with '\t'
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample next char
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update target sequence and states
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

# BLEU evaluation
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
smoothie = SmoothingFunction().method4

def evaluate_bleu(n=50):
    total_bleu = 0.0
    num_samples = min(n, len(input_texts))

    if num_samples == 0:
        print("No samples available for evaluation.")
        return

    for i in range(num_samples):
        input_seq = encoder_input_data[i:i+1]
        decoded_sentence = decode_sequence(input_seq)

        reference = [list(target_texts[i][1:-1])]  # Strip start and end tokens
        candidate = list(decoded_sentence.strip())

        bleu = sentence_bleu(reference, candidate, smoothing_function=smoothie)
        total_bleu += bleu

        print(f'Input: {input_texts[i]}')
        print(f'Target: {target_texts[i]}')
        print(f'Predicted: {decoded_sentence.strip()}')
        print(f'BLEU: {bleu:.4f}')
        print('---')

    avg_bleu = total_bleu / num_samples
    print(f'\nAverage BLEU score over {num_samples} samples: {avg_bleu:.4f}')

# Run evaluation
evaluate_bleu(n=50)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Input: hello
Target: 	bonjour

Predicted: bonjour
BLEU: 1.0000
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Input: hello
Target: 	bonjour

Predicted: bonjour
BLEU: 1.0000
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [ ]:
print("Number of input_texts:", len(input_texts))
print("Number of target_texts:", len(target_texts))
print("encoder_input_data shape:", encoder_input_data.shape)
print("decoder_input_data shape:", decoder_input_data.shape)
print("decoder_target_data shape:", decoder_target_data.shape)


Number of input_texts: 1000
Number of target_texts: 1000
encoder_input_data shape: (1000, 5, 4)
decoder_input_data shape: (1000, 9, 8)
decoder_target_data shape: (1000, 9, 8)


**Restore original texts from encoder input**

In [ ]:
# Reverse token indices
reversed_input_token_index = {i: char for char, i in input_token_index.items()}
reversed_target_token_index = {i: char for char, i in target_token_index.items()}

# Decode one-hot encoded sequence to text (input)
def decode_sequence_from_input(onehot_seq):
    indices = np.argmax(onehot_seq, axis=-1)  # Convert one-hot vectors to indices
    return ''.join(reversed_input_token_index.get(i, '') for i in indices if i > 0)

# Decode one-hot encoded sequence to text (target)
def decode_sequence_from_target(onehot_seq):
    indices = np.argmax(onehot_seq, axis=-1)
    return ''.join(reversed_target_token_index.get(i, '') for i in indices if i > 0)

# Rebuild input_texts and target_texts
input_texts = [decode_sequence_from_input(seq) for seq in encoder_input_data]
target_texts = ['\t' + decode_sequence_from_target(seq) + '\n' for seq in decoder_input_data]



In [ ]:
from keras.models import Model
from keras.layers import Input

# Define inference encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Define inference decoder model
decoder_state_input_h = Input(shape=(latent_dim,), name="decoder_input_h")
decoder_state_input_c = Input(shape=(latent_dim,), name="decoder_input_c")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Decoder input: one-hot like the training input
decoder_inputs_inf = Input(shape=(None, num_decoder_tokens), name="decoder_inputs_inf")

# Reuse decoder LSTM and Dense from training model
decoder_lstm_layer = model.get_layer('decoder_lstm')
decoder_dense_layer = model.get_layer('decoder_dense')

# LSTM outputs
decoder_lstm_outputs, state_h, state_c = decoder_lstm_layer(
    decoder_inputs_inf, initial_state=decoder_states_inputs
)
decoder_outputs_inf = decoder_dense_layer(decoder_lstm_outputs)
decoder_states = [state_h, state_c]

# Final decoder inference model
decoder_model = Model(
    [decoder_inputs_inf] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states
)




Model evaluation

In [ ]:
import json
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
from keras.utils import to_categorical
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Load JSON dataset
with open('your_eng.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract and preprocess input/output texts
input_texts = [item['source'] for item in data]
target_texts = ['\t' + item['target'] + '\n' for item in data]  # Start & end tokens

input_characters = sorted(set(char for text in input_texts for char in text))
target_characters = sorted(set(char for text in target_texts for char in text))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}
reverse_target_token_index = {i: char for char, i in target_token_index.items()}

# Vectorize data
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Model config
latent_dim = 128

# Encoder: Bidirectional LSTM
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_bi_lstm = Bidirectional(LSTM(latent_dim, return_state=True), name="bidirectional_lstm")
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bi_lstm(encoder_inputs)
state_h = Dense(latent_dim, activation="tanh")(Concatenate()([forward_h, backward_h]))
state_c = Dense(latent_dim, activation="tanh")(Concatenate()([forward_c, backward_c]))
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=32, epochs=10, validation_split=0.2)

# Inference models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_inf = Input(shape=(None, num_decoder_tokens))
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(decoder_inputs_inf, initial_state=decoder_states_inputs)
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)
decoder_model = Model([decoder_inputs_inf] + decoder_states_inputs, [decoder_outputs_inf, state_h_inf, state_c_inf])

# Decoding function
smoothing = SmoothingFunction().method4

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_token_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]

    return decoded_sentence

# Evaluate on all samples
for i in range(len(input_texts)):
    input_seq = encoder_input_data[i:i+1]
    decoded = decode_sequence(input_seq)
    print(f'Input: {input_texts[i]}')
    print(f'Target: {target_texts[i]}')
    print(f'Predicted: {decoded}')
    print('---')



FileNotFoundError: [Errno 2] No such file or directory: 'Assignment3Comp703.ipynb'

Stacked LSTM Model

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_lstm2 = LSTM(latent_dim, return_state=True)
encoder_outputs1, _, _ = encoder_lstm1(encoder_embedding)
encoder_outputs2, state_h, state_c = encoder_lstm2(encoder_outputs1)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs1, _, _ = decoder_lstm1(decoder_embedding, initial_state=encoder_states)
decoder_outputs2, _, _ = decoder_lstm2(decoder_outputs1)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs2)

# Define the model
stacked_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


Training

GRU

In [ ]:
from keras.models import Model
from keras.layers import Input, Concatenate

# Use correct layer names from your model
encoder_inputs_inf = Input(shape=(None,), name="encoder_inputs_inf")
encoder_embedding_layer = model.get_layer("embedding_7")  # Fixed name from your error message
encoder_embedding_inf = encoder_embedding_layer(encoder_inputs_inf)

encoder_bi_gru = model.get_layer("bidirectional_2")  # Also from your model's layer list
_, forward_h, backward_h = encoder_bi_gru(encoder_embedding_inf)

encoder_state_inf = Concatenate()([forward_h, backward_h])

encoder_model = Model(encoder_inputs_inf, encoder_state_inf)



In [ ]:
decoder_inputs_inf = Input(shape=(None,), name="decoder_inputs_inf")
decoder_state_input_h = Input(shape=(latent_dim * 2,), name="decoder_state_input_h")  # Match bidirectional size

decoder_embedding_layer = model.get_layer("embedding_8")
decoder_embedding_inf = decoder_embedding_layer(decoder_inputs_inf)

decoder_gru = model.get_layer("gru_2")
decoder_outputs, state_h = decoder_gru(decoder_embedding_inf, initial_state=decoder_state_input_h)

decoder_dense = model.get_layer("dense_4")
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs_inf, decoder_state_input_h], [decoder_outputs, state_h])


**Function (for Inference with GRUs)**

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

def decode_sequence(input_seq):
    # Encode the input sequence to get the initial state
    states_value = encoder_model.predict(input_seq)
    states_value = np.reshape(states_value, (1, -1))  # ensure shape (1, latent_dim*2)

    # Prepare the target sequence with the "start" token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']  # start token index

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        # Predict the next token and new state from the decoder
        output_tokens, h = decoder_model.predict([target_seq, states_value])

        # Sample the token with highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')

        decoded_sentence += sampled_char

        # Exit condition: either end token or max length reached
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update target sequence (next input to decoder)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update decoder states for next iteration
        states_value = np.reshape(h, (1, -1))

    return decoded_sentence





In [ ]:
'''for i in range(5):
    input_seq = encoder_input_data[i:i+1]
    decoded = decode_sequence(input_seq)
    print(f'Input: {input_texts[i]}')
    print(f'Target: {target_texts[i]}')
    print(f'Predicted: {decoded}')
    print('---')''''''


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━

**C**

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding

# Set your dimensions
latent_dim = 256  # You can adjust this
num_encoder_tokens = len(input_token_index)
num_decoder_tokens = len(target_token_index)

# Encoder
encoder_inputs = Input(shape=(None,), name="encoder_input")
encoder_embedding = Embedding(input_dim=num_encoder_tokens, output_dim=latent_dim, mask_zero=True, name="encoder_embedding")(encoder_inputs)

# Two LSTM layers
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True, name="encoder_lstm1")
encoder_outputs1, state_h1, state_c1 = encoder_lstm1(encoder_embedding)

encoder_lstm2 = LSTM(latent_dim, return_state=True, name="encoder_lstm2")
encoder_outputs2, state_h2, state_c2 = encoder_lstm2(encoder_outputs1)

encoder_states = [state_h2, state_c2]

# Decoder
decoder_inputs = Input(shape=(None,), name="decoder_input")
decoder_embedding = Embedding(input_dim=num_decoder_tokens, output_dim=latent_dim, mask_zero=True, name="decoder_embedding")(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax', name="dense")
decoder_outputs = decoder_dense(decoder_outputs)

# Full model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.summary()


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, None, 256) │     23,040 │ encoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_7         │ (None, None)      │          0 │ encoder_input[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm1       │ [(None, None,     │    525,312 │ encoder_embeddin… │
│ (LSTM)              │ 256), (None,      │            │ not_equal_7[0][0] │
│                     │ 256), (None,      │            │                   │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, None, 256) │     29,440 │ decoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm2       │ [(None, 256),     │    525,312 │ encoder_lstm1[0]… │
│ (LSTM)              │ (None, 256),      │            │ not_equal_7[0][0] │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    525,312 │ decoder_embeddin… │
│                     │ 256), (None,      │            │ encoder_lstm2[0]… │
│                     │ 256), (None,      │            │ encoder_lstm2[0]… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 115) │     29,555 │ decoder_lstm[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,657,971 (6.32 MB)

 Trainable params: 1,657,971 (6.32 MB)

 Non-trainable params: 0 (0.00 B)

**Train the Model**

In [ ]:
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=20,
    validation_split=0.2
)


Epoch 1/20
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 518s 174ms/step - loss: 2.0488 - val_loss: nan
Epoch 2/20
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 510s 172ms/step - loss: 1.2301 - val_loss: nan
Epoch 3/20
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 527s 160ms/step - loss: 1.0546 - val_loss: nan
Epoch 4/20
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 508s 171ms/step - loss: 0.9655 - val_loss: nan
Epoch 5/20
2973/2973 ━━━━━━━━━━━━━━━━━━━━ 562s 171ms/step - loss: 0.9091 - val_loss: nan
Epoch 6/20
1661/2973 ━━━━━━━━━━━━━━━━━━━━ 3:07 143ms/step - loss: 0.8719

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.utils import to_categorical

# Reduce complexity for faster training
latent_dim = 128  # Reduce from 256
num_samples = 1000  # Use only a small subset
epochs = 5  # Reduce training time
batch_size = 32

# Dummy data generation (Replace with your actual data loading)
input_texts = ["hello"] * num_samples
target_texts = ["\tbonjour\n"] * num_samples  # \t=start, \n=end

input_characters = sorted(list(set("".join(input_texts))))
target_characters = sorted(list(set("".join(target_texts))))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

# Vectorize
encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_encoder_tokens))
decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens))
decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens))

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define encoder with 2-layer LSTM
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs1, state_h1, state_c1 = encoder_lstm1(encoder_inputs)
encoder_lstm2 = LSTM(latent_dim, return_state=True)
encoder_outputs2, state_h2, state_c2 = encoder_lstm2(encoder_outputs1)

encoder_states = [state_h2, state_c2]

# Define decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile and train
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)


Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 87ms/step - accuracy: 0.2539 - loss: 1.7144 - val_accuracy: 0.7778 - val_loss: 0.9176
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.8137 - loss: 0.7116 - val_accuracy: 0.8889 - val_loss: 0.2875
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.8889 - loss: 0.2249 - val_accuracy: 0.8889 - val_loss: 0.1035
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.8889 - loss: 0.0884 - val_accuracy: 0.8889 - val_loss: 0.0582
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.8889 - loss: 0.0541 - val_accuracy: 0.8889 - val_loss: 0.0436


In [ ]:
# Rebuild encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Rebuild decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states_inf = [state_h_inf, state_c_inf]
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states_inf)


**decoding function**

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence with only the start character.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.  # Start character

    decoded_sentence = ''

    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n':
            break

        # Update the target sequence
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


**Test**

In [ ]:
for i in range(5):
    input_seq = encoder_input_data[i:i+1]
    decoded = decode_sequence(input_seq)
    print(f'Input: {input_texts[i]}')
    print(f'Target: {target_texts[i]}')
    print(f'Predicted: {decoded}')
    print('---')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Input: hello
Target: 	bonjour

Predicted: bonjour

---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Input: hello
Target: 	bonjour

Predicted: bonjour

---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━

**ncoder-Decoder with Bidirectional LSTM Encoder + LSTM Decoder python Copy Edit **

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Add
from keras.utils import to_categorical

# Parameters
latent_dim = 128
num_samples = 1000
epochs = 5
batch_size = 32

# Dummy data (replace with your actual data)
input_texts = ["hello"] * num_samples
target_texts = ["\tbonjour\n"] * num_samples  # \t=start, \n=end

input_characters = sorted(list(set("".join(input_texts))))
target_characters = sorted(list(set("".join(target_texts))))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

# Vectorize input and output
encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_bi_lstm = Bidirectional(LSTM(latent_dim, return_state=True, name="encoder_lstm"))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bi_lstm(encoder_inputs)

# Combine forward and backward states with Add layer, then Dense with tanh activation
state_h = Dense(latent_dim, activation="tanh")(Add()([forward_h, backward_h]))
state_c = Dense(latent_dim, activation="tanh")(Add()([forward_c, backward_c]))
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name="decoder_dense")
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile and train
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)




Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - accuracy: 0.2855 - loss: 1.6224 - val_accuracy: 0.7778 - val_loss: 0.7702
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.8501 - loss: 0.5986 - val_accuracy: 0.8889 - val_loss: 0.2085
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.8889 - loss: 0.1578 - val_accuracy: 0.8889 - val_loss: 0.0709
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.8889 - loss: 0.0614 - val_accuracy: 0.8889 - val_loss: 0.0437
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.8889 - loss: 0.0408 - val_accuracy: 0.8889 - val_loss: 0.0349


In [ ]:
from keras.models import Model
from keras.layers import Input
import numpy as np

# --- Encoder inference model ---
encoder_model = Model(encoder_inputs, encoder_states)

# --- Decoder inference model ---
decoder_state_input_h = Input(shape=(latent_dim,), name="decoder_state_input_h")
decoder_state_input_c = Input(shape=(latent_dim,), name="decoder_state_input_c")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# --- Decoding function ---
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1 with only the start character.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Initialize variables
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character.
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

# --- Test decoding on some samples ---
for seq_index in range(5):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(f"Input: {input_texts[seq_index]}")
    print(f"Predicted translation: {decoded_sentence}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Input: hello
Predicted translation: bonjour

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Input: hello
Predicted translation: bonjour

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/s

**Testing Code**

In [ ]:
num_test_samples = 10  # Or size of your test set

correct = 0
for i in range(num_test_samples):
    input_seq = encoder_input_data[i:i+1]  # Or your test input data here
    decoded_sentence = decode_sequence(input_seq)

    print(f"Input: {input_texts[i]}")
    print(f"Target: {target_texts[i].strip()}")
    print(f"Predicted: {decoded_sentence.strip()}")
    print("---")

    if decoded_sentence.strip() == target_texts[i].strip():
        correct += 1

accuracy = correct / num_test_samples
print(f"Exact match accuracy on test set: {accuracy*100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Input: hello
Target: bonjour
Predicted: bonjour
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Input: hello
Target: bonjour
Predicted: bonjour
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━

**Calculate BLEU Score**

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

bleu_scores = []

for i in range(num_test_samples):
    input_seq = encoder_input_data[i:i+1]
    decoded_sentence = decode_sequence(input_seq).strip()
    reference = [list(target_texts[i].strip())]  # Reference as list of chars or words
    candidate = list(decoded_sentence)
    bleu = sentence_bleu(reference, candidate)
    bleu_scores.append(bleu)

avg_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU score: {avg_bleu:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━

**testing and evaluation code snippet**

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# Number of test samples to evaluate
num_test_samples = 10  # adjust as needed, or use your test set size

correct = 0
bleu_scores = []

for i in range(num_test_samples):
    input_seq = encoder_input_data[i:i+1]
    decoded_sentence = decode_sequence(input_seq).strip()
    target_sentence = target_texts[i].strip()

    print(f"Input: {input_texts[i]}")
    print(f"Target: {target_sentence}")
    print(f"Predicted: {decoded_sentence}")
    print("---")

    # Exact match accuracy
    if decoded_sentence == target_sentence:
        correct += 1

    # Calculate BLEU score (character-level)
    reference = [list(target_sentence)]
    candidate = list(decoded_sentence)
    bleu = sentence_bleu(reference, candidate)
    bleu_scores.append(bleu)

accuracy = correct / num_test_samples * 100
avg_bleu = sum(bleu_scores) / len(bleu_scores)

print(f"Exact match accuracy: {accuracy:.2f}%")
print(f"Average BLEU score: {avg_bleu:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Input: hello
Target: bonjour
Predicted: bonjour
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Input: hello
Target: bonjour
Predicted: bonjour
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving fra.txt to fra.txt


In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from sklearn.model_selection import train_test_split

# Load the uploaded file
import io

# Pick the uploaded filename dynamically
filename = list(uploaded.keys())[0]
with io.open(filename, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# -------- Prepare data --------
num_samples = 1000  # adjust as needed

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for line in lines[:num_samples]:
    parts = line.strip().split('\t')
    if len(parts) < 2:
        continue
    input_text, target_text = parts[0], parts[1]
    target_text = '\t' + target_text + '\n'  # Start & end tokens
    input_texts.append(input_text)
    target_texts.append(target_text)
    input_characters.update(list(input_text))
    target_characters.update(list(target_text))

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

# -------- Vectorization --------
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# -------- Model (BiLSTM encoder + LSTM decoder) --------
latent_dim = 256

encoder_inputs = Input(shape=(None, num_encoder_tokens), name="encoder_inputs")
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="encoder_lstm")
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

from keras.layers import Bidirectional
encoder_bi = Bidirectional(encoder_lstm, name="bidirectional_lstm")
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bi(encoder_inputs)
state_h = Dense(latent_dim, activation="tanh")(forward_h + backward_h)
state_c = Dense(latent_dim, activation="tanh")(forward_c + backward_c)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens), name="decoder_inputs")
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name="decoder_dense")
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# -------- Train --------
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=20,
    validation_split=0.2
)


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.0312 - loss: 1.6772 - val_accuracy: 0.0574 - val_loss: 1.5743
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0593 - loss: 1.3798 - val_accuracy: 0.0614 - val_loss: 1.5005
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0631 - loss: 1.3222 - val_accuracy: 0.0602 - val_loss: 1.4896
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0645 - loss: 1.3216 - val_accuracy: 0.0614 - val_loss: 1.4768
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0661 - loss: 1.2947 - val_accuracy: 0.0635 - val_loss: 1.4757
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0744 - loss: 1.2958 - val_accuracy: 0.0617 - val_loss: 1.4710
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0716 - loss: 1.3068 - val_accuracy: 0.0606 - val_loss: 1.4693
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0759 - loss: 1.2858 - val_accuracy: 0.0608 - v

In [ ]:
# prompt: save the model trained in the above cell and download it

model_name = "blstm_seq2seq_model.h5"
model.save(model_name)

# Download the saved model file
files.download(model_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
'''import numpy as np
import pickle
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, dot, concatenate, Softmax
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu
import io

# ---------- Load & Prepare Data ----------
filename = list(uploaded.keys())[0]
with io.open(filename, 'r', encoding='utf-8') as f:
    lines = f.readlines()

num_samples = 1000
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for line in lines[:num_samples]:
    parts = line.strip().split('\t')
    if len(parts) < 2:
        continue
    input_text, target_text = parts[0], parts[1]
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    input_characters.update(list(input_text))
    target_characters.update(list(target_text))

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# ---------- Define the Model ----------
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens), name="encoder_inputs")
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_bi = Bidirectional(encoder_lstm, name="bidirectional_lstm")
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bi(encoder_inputs)
state_h = Dense(latent_dim, activation="tanh")(forward_h + backward_h)
state_c = Dense(latent_dim, activation="tanh")(forward_c + backward_c)
encoder_states = [state_h, state_c]

# Project encoder outputs to match decoder dimension
encoder_proj_outputs = Dense(latent_dim, activation="tanh", name="encoder_proj")(encoder_outputs)

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens), name="decoder_inputs")
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Attention mechanism
attention_scores = dot([decoder_outputs, encoder_proj_outputs], axes=[2, 2])
attention_weights = Softmax(axis=-1)(attention_scores)
context = dot([attention_weights, encoder_proj_outputs], axes=[2, 1])
decoder_combined_context = concatenate([context, decoder_outputs])

# Output layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name="decoder_dense")
decoder_outputs = decoder_dense(decoder_combined_context)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# ---------- Train the Model ----------
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=30,
          validation_split=0.2)

# ---------- Save the Model and Token Indexes ----------
model.save('seq2seq_training_model.h5')
with open('token_indices.pkl', 'wb') as f:
    pickle.dump({
        'input_token_index': input_token_index,
        'target_token_index': target_token_index,
        'reverse_target_char_index': reverse_target_char_index
    }, f)

# ---------- Inference Setup ----------
encoder_model = Model(encoder_inputs, [encoder_proj_outputs, state_h, state_c])

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
encoder_proj_inf = Input(shape=(None, latent_dim))

decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
    decoder_inputs, initial_state=[decoder_state_input_h, decoder_state_input_c])

attention_scores_inf = dot([decoder_outputs_inf, encoder_proj_inf], axes=[2, 2])
attention_weights_inf = Softmax(axis=-1)(attention_scores_inf)
context_inf = dot([attention_weights_inf, encoder_proj_inf], axes=[2, 1])
decoder_combined_context_inf = concatenate([context_inf, decoder_outputs_inf])
decoder_outputs_inf = decoder_dense(decoder_combined_context_inf)

decoder_model = Model(
    [decoder_inputs, decoder_state_input_h, decoder_state_input_c, encoder_proj_inf],
    [decoder_outputs_inf, state_h_inf, state_c_inf]
)

# ---------- Decoding Function ----------
def decode_sequence(input_seq):
    encoder_outs, state_h, state_c = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    decoded_sentence = ''

    while True:
        output_tokens, h, c = decoder_model.predict(
            [target_seq, state_h, state_c, encoder_outs])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            break
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        state_h, state_c = h, c
    return decoded_sentence

# ---------- Evaluate the Model ----------
num_test_samples = 10
correct = 0
bleu_scores = []

for i in range(num_test_samples):
    input_seq = encoder_input_data[i:i+1]
    decoded_sentence = decode_sequence(input_seq).strip()
    target_sentence = target_texts[i].strip()

    print(f"Input: {input_texts[i]}")
    print(f"Target: {target_sentence}")
    print(f"Predicted: {decoded_sentence}")
    print("---")

    if decoded_sentence == target_sentence:
        correct += 1

    reference = [list(target_sentence)]
    candidate = list(decoded_sentence)
    bleu = sentence_bleu(reference, candidate)
    bleu_scores.append(bleu)

accuracy = correct / num_test_samples * 100
avg_bleu = sum(bleu_scores) / len(bleu_scores)

print(f"Exact match accuracy: {accuracy:.2f}%")
print(f"Average BLEU score: {avg_bleu:.4f}")'''

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from google.colab import files

# Upload the file manually via Colab file dialog
uploaded = files.upload()

# Read and preprocess the uploaded file
file_name = list(uploaded.keys())[0]
with open(file_name, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Extract input-output pairs (first two tab-separated fields)
input_texts = []
target_texts = []
num_samples = 10000  # Increase sample size

for line in lines[:num_samples]:
    parts = line.strip().split('\t')
    if len(parts) >= 2:
        input_texts.append(parts[0])
        target_texts.append('\t' + parts[1] + '\n')  # Start and end tokens

# Token indexing
input_characters = sorted(set(''.join(input_texts)))
target_characters = sorted(set(''.join(target_texts)))

input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

# Vectorize input and output
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Model architecture
latent_dim = 256

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_bi_lstm = Bidirectional(LSTM(latent_dim, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bi_lstm(encoder_inputs)
state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=50,
    validation_split=0.2
)

# Inference models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim * 2,))
decoder_state_input_c = Input(shape=(latent_dim * 2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + [state_h, state_c]
)

# Inference function
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            break

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]

    return decoded_sentence

# BLEU evaluation
smoothie = SmoothingFunction().method4
total_bleu = 0

for i in range(50):
    input_seq = encoder_input_data[i:i+1]
    decoded = decode_sequence(input_seq)
    ref = [list(target_texts[i][1:-1])]
    hyp = list(decoded.strip())
    bleu = sentence_bleu(ref, hyp, smoothing_function=smoothie)
    total_bleu += bleu
    print(f"Input: {input_texts[i]}\nTarget: {target_texts[i]}\nPredicted: {decoded}\nBLEU: {bleu:.4f}\n---")

print(f"\nAverage BLEU score over 50 samples: {total_bleu/50:.4f}")



Saving fra.txt to fra (1).txt
Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.0406 - loss: 1.0648 - val_accuracy: 0.0574 - val_loss: 1.0360
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.0501 - loss: 0.9306 - val_accuracy: 0.0709 - val_loss: 0.9979
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.0611 - loss: 0.8975 - val_accuracy: 0.0784 - val_loss: 0.9623
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.0711 - loss: 0.8698 - val_accuracy: 0.0890 - val_loss: 0.9495
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 211s 1s/step - accuracy: 0.0781 - loss: 0.8445 - val_accuracy: 0.0984 - val_loss: 0.9121
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.0873 - loss: 0.8217 - val_accuracy: 0.1063 - val_loss: 0.8806
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.0942 - loss: 0.7853 - val_accuracy: 0.1128 - val_loss: 0.8545
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 211s 1s/step - accuracy: 0.